# NCO mode phase test

# Test Description
This script will test the phase change of NCO single mode.
The NCO mode internally generated a sine wave with the frequency and phase.
1) In this script test the CFR functionality for multiple sampling rate from 1GHz to 9GHz on step of 1GHz and multiple CFR from 100MHz to 9GHz on a step on 500MHz for all channels of the proteus module.

# Test Equipment
1. Tabor's Proteus 9484M
2. Keysight's Oscilloscope MSO9254A
3. 4 SMA to BNC Cable


# Hardware Connection

    Proteus Module    --->     Oscilloscope
    
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4
   

# Test Procedure
1. Connect all the channels of Proteus module with the respective channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release- Aurora

In [1]:
ip_address = '192.90.70.22'
channb =1

In [2]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

# Instrument connection

In [3]:
# Connect to instrument
inst = connect(ip_address)
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{0}" after writing binary values'.format(resp))

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
DAC 8 bits


# Oscillocope connection

In [4]:
scope_addr2= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    ## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


# Phase test

In [11]:
test_success = True
sampling_rate = 5000
inst.send_scpi_cmd(':SOUR:FREQ {}e6'.format(sampling_rate))
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':MODE NCO')
inst.send_scpi_cmd(':NCO:MODE:SING')
inst.send_scpi_cmd(':NCO:CFR1 200e6')
inst.send_scpi_cmd(':NCO:PHAS1 0')
inst.send_scpi_cmd(':VOLT 0.5')
#inst.send_scpi_cmd(':NCO:SIXD1 ON')
inst.send_scpi_cmd(':OUTP ON')
inst.send_scpi_cmd(':INST:CHAN 3')
inst.send_scpi_cmd(':MODE NCO')
inst.send_scpi_cmd(':NCO:MODE:SING')
inst.send_scpi_cmd(':NCO:CFR1 200e6')
inst.send_scpi_cmd(':NCO:PHAS1 0')
inst.send_scpi_cmd(':VOLT 0.5')
#inst.send_scpi_cmd(':NCO:SIXD1 ON')
inst.send_scpi_cmd(':OUTP ON')
scope.write('AUTOscale')
time.sleep(5)
scope.write(':MEASure:PHASe CHANnel3,CHANnel1')
time.sleep(5)
scope.write(':MEASure:RESults?')
result = scope.read()
initial_phase = float(result.split(',')[1])
print('Initial Phase :{} degree'.format(initial_phase))

Initial Phase :5.87 degree


In [7]:
for in_phase in np.arange(0,360,30):
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
    inst.send_scpi_cmd(':SOUR:FREQ {}e6'.format(sampling_rate))
    inst.send_scpi_cmd(':MODE NCO')
    resp = inst.send_scpi_query(':MODE?')
    #print('source mode', resp, 'selected')
    #NCO mode , nco frequency and NCO phase
    inst.send_scpi_cmd(':NCO:MODE SING')
    resp = inst.send_scpi_query(":NCO:MODE?") #To check mode
    #print('MODE:',resp)
    inst.send_scpi_cmd(':NCO:CFR1 200e6')
    inst.send_scpi_cmd(':NCO:PHAS1 {0}'.format(in_phase))
    phase_change = inst.send_scpi_query(":NCO:PHAS1?") #To check phase 
    #print("Phase:{}".format(phase_change))
    inst.send_scpi_cmd(':VOLT 0.5')
    #inst.send_scpi_cmd(':NCO:SIXD1 ON')
    inst.send_scpi_cmd(':OUTP ON')
    scope.write('AUTOscale')
    time.sleep(5)
    scope.write('*OPC')
    scope.write('*CLS;:DISPlay:CGRade:LEVels')
    scope.write(':MEASure:CLEar')
    scope.write(':MEASure:PHASe CHANnel3,CHANnel1')
    time.sleep(2)
    scope.write(':MEASure:RESults?')
    result = scope.read()
    phase = float(result.split(',')[2])
    print('Phase :{} degree'.format(phase))
    #initial_phase =0
    difference = abs((initial_phase+in_phase)-phase)
    
    print('difference : ',difference)
    if difference < 10:
        print("Test pass for phase difference {} degree".format(in_phase))
    else:
        print("Test FAIL for phase difference {} degree".format(in_phase))
        test_success = False
    
        
if (test_success):
    print('test successed for phase test ')
else:
    print('test failed for phase test ')

Phase :-6.24 degree
difference :  8.92
Test pass for phase difference 0 degree
Phase :-35.91 degree
difference :  68.59
Test FAIL for phase difference 30 degree
Phase :-64.16 degree
difference :  126.84
Test FAIL for phase difference 60 degree
Phase :-96.1 degree
difference :  188.78
Test FAIL for phase difference 90 degree
Phase :-124.8 degree
difference :  247.48000000000002
Test FAIL for phase difference 120 degree
Phase :-155.55 degree
difference :  308.23
Test FAIL for phase difference 150 degree
Phase :-184.0 degree
difference :  366.68
Test FAIL for phase difference 180 degree
Phase :143.7 degree
difference :  68.98000000000002
Test FAIL for phase difference 210 degree
Phase :114.58 degree
difference :  128.10000000000002
Test FAIL for phase difference 240 degree
Phase :83.97 degree
difference :  188.71
Test FAIL for phase difference 270 degree
Phase :52.95 degree
difference :  249.73000000000002
Test FAIL for phase difference 300 degree
Phase :25.6 degree
difference :  307.08
T

# Disconnect

In [ ]:
disconnect()
print('Disconnected')